#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [1]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import datetime
import os


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

LOG_NAME = 'log.csv'

server_stats = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler,
                            allow_none=True)


# Добавление строки в лог. max_log_length - максимальное количество записей журнала. 0 - нет ограничения
def add_log(sname, execution_duration_ms, max_log_length):
    if max_log_length > 0:
        with open(LOG_NAME) as f:
            full_log = f.readlines()
        
        # если есть ограничение, и оно превышено
        if len(full_log) >= max_log_length:
            filename = f'{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}log.csv'
            
            # сохранение старого лога и обнуление файла
            with open(filename, 'a') as f_save:
                f_save.writelines(full_log)
            with open(LOG_NAME, 'w'):
                pass
    with open(LOG_NAME, 'a') as f:
        f.write(f'{str(sname)}, {execution_duration_ms:2f}ms, {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n')
server_stats.register_function(add_log, 'add_log')

def get_log_records(events: list[str], datetime_start: str, datetime_end: str):
    def str_date_to_serverdate(date: str) -> str:
        result = date.replace('-', '').replace(' ', '_').replace(':', '')
        return result

    log_files_names = filter(lambda n: n.endswith(".csv"), os.listdir())
    readed_files_names = filter(lambda f: datetime_start <= str_date_to_serverdate(f.split("log")[0]) <= datetime_end, log_files_names)
    records = []
    for f in readed_files_names:
        with open(f) as f:
            records += [r.split("\n")[0] for r in f.readlines()]
    return [rec for rec in filter(
        lambda r: r.split(',')[0] in events and datetime_start <= r.split(',')[1].strip() <= datetime_end
        , records)]
server_stats.register_function(get_log_records, 'get_log_records')
    

print ("Listening on port 8018...")
server_stats.serve_forever()


Listening on port 8018...


127.0.0.1 - - [25/Oct/2023 16:17:02] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:05] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:08] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:11] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:14] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:17] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:20] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:23] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:26] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 16:17:29] "POST /RPC2 HTTP/1.1" 200 -


KeyboardInterrupt: 

In [3]:
get_log_records(["pow", "sum"], "2023-09-12 18:07", "3000")

[]